In [1]:
file_path = "C:/Users/gch05/data_min/dataset/"

In [2]:
#Importing librairies
import pandas as pd
import numpy as np
 
# Scikit-learn library: For SVM
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix
from sklearn import svm

import itertools

# Matplotlib library to plot the charts
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab

# Library for the statistic data vizualisation
import seaborn as sns

%matplotlib inline

In [3]:
# 데이터 불러오기
data = pd.read_csv(file_path + 'pca_train_test.csv')
data.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,target
0,-0.186803,-1.000143,0.745271,0.396115,-1.285372,-0.708153,0.114003,0.325531,0.592047,0.439203,...,0.884376,-0.557766,0.036501,0.809532,0.478148,0.135564,1.472248,0.428958,-0.381381,0
1,-0.705777,-0.671973,0.878802,0.384820,0.332147,0.265318,0.581809,-0.587116,0.875597,-0.961119,...,-0.536303,-0.159865,0.910140,0.123068,-0.891751,0.650127,0.867556,-0.841684,0.475563,0
2,3.973764,0.752199,-0.057570,0.722408,1.857562,-1.213780,0.526862,-1.335082,0.488178,-0.693094,...,0.431250,0.468316,0.554997,0.719100,-0.347534,0.322468,-0.220685,0.436515,-0.433111,0
3,1.025226,0.970736,0.634796,-0.730365,-0.767119,0.839303,0.244287,0.100426,0.462459,0.848337,...,-0.503240,0.372951,0.879522,0.369692,-0.584512,1.226589,-0.281516,-0.572916,0.434625,0
4,-0.892340,-0.479857,0.500884,-1.393628,0.103438,0.917956,-1.092323,0.107218,0.561216,0.387365,...,-0.379418,0.749823,0.204488,0.400505,0.618971,0.204516,-0.777073,0.554618,-0.069441,0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284806 entries, 0 to 284805
Data columns (total 29 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   V1      284806 non-null  float64
 1   V2      284806 non-null  float64
 2   V3      284806 non-null  float64
 3   V4      284806 non-null  float64
 4   V5      284806 non-null  float64
 5   V6      284806 non-null  float64
 6   V7      284806 non-null  float64
 7   V8      284806 non-null  float64
 8   V9      284806 non-null  float64
 9   V10     284806 non-null  float64
 10  V11     284806 non-null  float64
 11  V12     284806 non-null  float64
 12  V13     284806 non-null  float64
 13  V14     284806 non-null  float64
 14  V15     284806 non-null  float64
 15  V16     284806 non-null  float64
 16  V17     284806 non-null  float64
 17  V18     284806 non-null  float64
 18  V19     284806 non-null  float64
 19  V20     284806 non-null  float64
 20  V21     284806 non-null  float64
 21  V22     28

In [5]:
data.shape

(284806, 29)

# 전처리

## 1) 중복 제거

In [6]:
# 중복된 row 이는지 확인
print(f"Total duplicate rows = {data.duplicated().sum()}")
print(f"Data shape before removing duplicates = {data.shape}")

Total duplicate rows = 0
Data shape before removing duplicates = (284806, 29)


In [7]:
# 중복된 행 제거
data_cleaned = data.drop_duplicates()

# 중복 제거 후의 데이터셋 크기 확인
print(f"Data shape after removing duplicates = {data_cleaned.shape}")

Data shape after removing duplicates = (284806, 29)


## 데이터 불균형 처리 - SMOTE

In [8]:
!pip install imbalanced-learn

In [9]:
data_cleaned['target'].value_counts()

target
0    284314
1       492
Name: count, dtype: int64

In [10]:
y = data_cleaned['target']
X = data_cleaned.drop(['target'], axis = 1)
print(X.info())
print(y)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284806 entries, 0 to 284805
Data columns (total 28 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   V1      284806 non-null  float64
 1   V2      284806 non-null  float64
 2   V3      284806 non-null  float64
 3   V4      284806 non-null  float64
 4   V5      284806 non-null  float64
 5   V6      284806 non-null  float64
 6   V7      284806 non-null  float64
 7   V8      284806 non-null  float64
 8   V9      284806 non-null  float64
 9   V10     284806 non-null  float64
 10  V11     284806 non-null  float64
 11  V12     284806 non-null  float64
 12  V13     284806 non-null  float64
 13  V14     284806 non-null  float64
 14  V15     284806 non-null  float64
 15  V16     284806 non-null  float64
 16  V17     284806 non-null  float64
 17  V18     284806 non-null  float64
 18  V19     284806 non-null  float64
 19  V20     284806 non-null  float64
 20  V21     284806 non-null  float64
 21  V22     28

In [11]:
!pip install scikit-learn

In [12]:
from imblearn.over_sampling import SMOTE
# SMOTE 객체 생성
smote = SMOTE(random_state=42)

# 데이터 업샘플링 수행
X_resampled, y_resampled = smote.fit_resample(X, y)

# 업샘플링된 데이터의 클래스 분포 확인
resampled_class_distribution = pd.Series(y_resampled).value_counts()

resampled_class_distribution

target
0    284314
1    284314
Name: count, dtype: int64

In [13]:
data_cleaned = pd.concat([X_resampled, y_resampled], axis=1)
data_cleaned.shape

(568628, 29)

In [14]:
data_cleaned.shape

(568628, 29)

# Modeling

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
SEED = 42

# Split the data into features and target variable
X = data_cleaned.drop(columns=[ 'target'])  # Features
y = data_cleaned['target']
                   # Target variable
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# 비선형 SVM 분류기 초기화 (RBF 커널 사용)
svm_model = SVC(kernel='rbf', random_state=SEED)

# 모델 훈련
svm_model.fit(X_train, y_train)

# 검증 세트에 대한 예측 수행
y_val_pred = svm_model.predict(X_val)

# 검증 세트에 대한 모델 성능 평가
val_accuracy = accuracy_score(y_val, y_val_pred)
val_classification_report = classification_report(y_val, y_val_pred)

print("Validation Accuracy:", val_accuracy)
print("Validation Classification Report:\n", val_classification_report)



Validation Accuracy: 0.9964124298753143
Validation Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.99      1.00     56794
           1       0.99      1.00      1.00     56932

    accuracy                           1.00    113726
   macro avg       1.00      1.00      1.00    113726
weighted avg       1.00      1.00      1.00    113726



In [16]:
print(val_accuracy)
print('---------------------------------------------------------')
print(val_classification_report)

0.9964124298753143
---------------------------------------------------------
              precision    recall  f1-score   support

           0       1.00      0.99      1.00     56794
           1       0.99      1.00      1.00     56932

    accuracy                           1.00    113726
   macro avg       1.00      1.00      1.00    113726
weighted avg       1.00      1.00      1.00    113726



# 예닮_Data scaling

In [15]:
data_cleaned_yd = data_cleaned.copy()

In [17]:
from sklearn.preprocessing import MinMaxScaler
#꼬리가 오른쪽으로 긴 변수
pro_skew = ['V3', 'V5', 'V7','V9', 'V10', 'V12', 'V14', 'V15', 'V16', 'V17', 'V18', 'V24','V27', 'V28']
neg_skew = ['V2', 'V11', 'V19', 'V20', 'V21']
normal_col = ['V1', 'V4','V6', 'V8', 'V13', 'V22', 'V23', 'V25', 'V26' ]

# pro_skew 로그변환 적용
# 변수 내의 최소값이 0 이하인 경우, 모든 값을 조금 올려서 로그 변환이 가능하도록 조정
if (data_cleaned_yd[pro_skew].min() <= 0).any():  # .any()를 추가하여 Series 전체에서 하나라도 조건을 만족하는지 확인
    for col in pro_skew:
        if data_cleaned_yd[col].min() <= 0:
            data_cleaned_yd[col] += abs(data_cleaned_yd[col].min()) + 0.1

for col in pro_skew:
    data_cleaned_yd[col + '_log_transformed'] = np.log(data_cleaned_yd[col])


# neg_skew는 음의 왜도.
# 제곱 변환 적용
for col in neg_skew:
  data_cleaned[col + '_squared_transformed'] = data_cleaned_yd[col] ** 2

#normal_col
# 정규화할 변수 리스트
scaler = MinMaxScaler()
for col in normal_col:
    data_cleaned_yd[col + '_minmax_transformed'] = scaler.fit_transform(data_cleaned_yd[col].values.reshape(-1, 1)) # 2차원 배열로 바꿔서 실행


# 적용 다한 column은 drop
data_cleaned_yd.drop(columns = pro_skew + normal_col + neg_skew, inplace = True)

data_cleaned_yd.head()

,target,V3_log_transformed,V5_log_transformed,V7_log_transformed,V9_log_transformed,V10_log_transformed,V12_log_transformed,V14_log_transformed,V15_log_transformed,V16_log_transformed,...,V21_squared_transformed,V1_minmax_transformed,V4_minmax_transformed,V6_minmax_transformed,V8_minmax_transformed,V13_minmax_transformed,V22_minmax_transformed,V23_minmax_transformed,V25_minmax_transformed,V26_minmax_transformed
0,0,3.497852,3.317053,3.211946,3.005139,3.224547,2.915893,2.474494,3.507461,2.918635,...,0.311103,0.461003,0.347621,0.397408,0.471259,0.465575,0.457256,0.398142,0.455267,0.430776
1,0,3.501885,3.374048,3.230613,3.019086,3.167239,2.803769,2.656599,3.554492,2.910527,...,0.025557,0.450439,0.347493,0.409417,0.455616,0.470091,0.468751,0.388390,0.468648,0.415897
2,0,3.473256,3.424976,3.228439,2.999981,3.178465,2.845195,2.749169,3.506808,2.906336,...,0.219320,0.545693,0.351312,0.391170,0.442796,0.463943,0.464078,0.396857,0.460127,0.389120
3,0,3.494503,3.335671,3.217180,2.998699,3.240689,2.804290,2.674310,3.506606,2.901787,...,0.139092,0.485674,0.334878,0.416498,0.467401,0.463254,0.468348,0.391894,0.483639,0.387623
4,0,3.490429,3.366184,3.162138,3.003610,3.222483,2.866141,2.747091,3.541557,2.848338,...,0.562234,0.446641,0.327375,0.417469,0.467517,0.465363,0.459466,0.392331,0.457060,0.375430


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
SEED = 42

# Split the data into features and target variable
X_yd = data_cleaned_yd.drop(columns=[ 'target'])  # Features
y_yd = data_cleaned_yd['target']
                   # Target variable
X_train_yd, X_val_yd, y_train_yd, y_val_yd = train_test_split(X_yd, y_yd, test_size=0.2, random_state=42)

# 비선형 SVM 분류기 초기화 (RBF 커널 사용)
svm_model_yd = SVC(kernel='rbf', random_state=SEED)

# 모델 훈련
svm_model_yd.fit(X_train_yd, y_train_yd)

In [20]:
# 검증 세트에 대한 예측 수행
y_val_pred_yd = svm_model_yd.predict(X_val_yd)

# 검증 세트에 대한 모델 성능 평가
val_accuracy_yd = accuracy_score(y_val_yd, y_val_pred_yd)
val_classification_report_yd = classification_report(y_val_yd, y_val_pred_yd)

print("Validation Accuracy:", val_accuracy_yd)
print("Validation Classification Report:\n", val_classification_report_yd)


Validation Accuracy: 0.6541160332729543
Validation Classification Report:
               precision    recall  f1-score   support

           0       0.69      0.57      0.62     56794
           1       0.63      0.74      0.68     56932

    accuracy                           0.65    113726
   macro avg       0.66      0.65      0.65    113726
weighted avg       0.66      0.65      0.65    113726



In [21]:
print(val_accuracy_yd)
print('---------------------------------------------------------')
print(val_classification_report_yd)

0.6541160332729543
---------------------------------------------------------
              precision    recall  f1-score   support

           0       0.69      0.57      0.62     56794
           1       0.63      0.74      0.68     56932

    accuracy                           0.65    113726
   macro avg       0.66      0.65      0.65    113726
weighted avg       0.66      0.65      0.65    113726



# 진기 Scaling

In [15]:
data_cleaned_jg = data_cleaned.copy()

In [16]:
from sklearn.preprocessing import MinMaxScaler, QuantileTransformer
import pandas as pd
import numpy as np

def quantileTransform(data, columns):
    qt = QuantileTransformer()
    scaled_data = pd.DataFrame(qt.fit_transform(data[columns]), columns=columns)
    return scaled_data

def boxcox_transform(data, columns, lambda_value):
    if lambda_value == 0:
        return np.log(data[columns])
    else:
        return (data[columns] ** lambda_value - 1) / lambda_value


left_skewed_cols = ['V3', 'V5', 'V7', 'V9', 'V10', 'V12', 'V14', 'V15', 'V16', 'V17', 'V18', 'V24', 'V27', 'V28']
right_skewed_cols = ['V2', 'V11', 'V19', 'V20', 'V21']


if (data_cleaned_jg[left_skewed_cols].min(axis=0) <= 0).any():
    for col in left_skewed_cols:
        if data_cleaned_jg[col].min() <= 0:
            data_cleaned_jg[col] += abs(data_cleaned_jg[col].min()) + 0.1

# train_scaler
train_left_skewed = boxcox_transform(data_cleaned_jg, left_skewed_cols)
train_right_skewed = quantileTransform(data_cleaned_jg, right_skewed_cols)

train_transformed_jg = pd.concat([data_cleaned_jg, train_left_skewed.add_suffix('_log_Transform'), train_right_skewed.add_suffix('_qt_Transform')], axis=1)

# 그 외 수치
# minmax? standard?
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.preprocessing import StandardScaler
s_columns = ['V1', 'V4', 'V6', 'V8', 'V13', 'V22', 'V23', 'V25', 'V26']
standardScaler = StandardScaler()

for col in s_columns:
    train_transformed_sm[col + '_standard_Transform'] = standardScaler.fit_transform(train_transformed_sm[[col]])

In [20]:
Train_jg = train_transformed_jg.copy()
Train_jg.drop(columns = left_skewed_cols + right_skewed_cols + s_columns, inplace = True)
Train_jg.head()

,target,V3_log_Transform,V5_log_Transform,V7_log_Transform,V9_log_Transform,V10_log_Transform,V12_log_Transform,V14_log_Transform,V15_log_Transform,V16_log_Transform,...,V21_qt_Transform,V1_MM_Transform,V4_MM_Transform,V6_MM_Transform,V8_MM_Transform,V13_MM_Transform,V22_MM_Transform,V23_MM_Transform,V25_MM_Transform,V26_MM_Transform
0,0,3.527665,3.352671,3.251434,3.053483,3.263550,2.968633,2.555342,3.536994,2.971235,...,0.195995,0.461003,0.347621,0.397408,0.471259,0.465575,0.457256,0.398142,0.455267,0.430776
1,0,3.531580,3.407726,3.269385,3.066776,3.208496,2.862586,2.724432,3.582686,2.963544,...,0.295873,0.450439,0.347493,0.409417,0.455616,0.470091,0.468751,0.388390,0.468648,0.415897
2,0,3.503800,3.457007,3.267293,3.048569,3.219271,2.901692,2.811186,3.536359,2.959569,...,0.671576,0.545693,0.351312,0.391170,0.442796,0.463943,0.464078,0.396857,0.460127,0.389120
3,0,3.524415,3.370643,3.256466,3.047348,3.279079,2.863078,2.740991,3.536163,2.955257,...,0.561215,0.485674,0.334878,0.416498,0.467401,0.463254,0.468348,0.391894,0.483639,0.387623
4,0,3.520461,3.400123,3.203601,3.052027,3.261565,2.921498,2.809234,3.570114,2.904662,...,0.892598,0.446641,0.327375,0.417469,0.467517,0.465363,0.459466,0.392331,0.457060,0.375430


In [22]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
SEED = 42

# Split the data into features and target variable
X_jg = Train_jg.drop(columns=[ 'target'])  # Features
y_jg = Train_jg['target']
                   # Target variable
X_train_jg, X_val_jg, y_train_jg, y_val_jg = train_test_split(X_jg, y_jg, test_size=0.2, random_state=42)

# 비선형 SVM 분류기 초기화 (RBF 커널 사용)
svm_model_jg = SVC(kernel='rbf', random_state=SEED)

# 모델 훈련
svm_model_jg.fit(X_train_jg, y_train_jg)

# 검증 세트에 대한 예측 수행
y_val_pred_jg = svm_model_jg.predict(X_val_jg)

# 검증 세트에 대한 모델 성능 평가
val_accuracy_jg = accuracy_score(y_val_jg, y_val_pred_jg)
val_classification_report_jg = classification_report(y_val_jg, y_val_pred_jg)

print("Validation Accuracy:", val_accuracy_jg)
print("Validation Classification Report:\n", val_classification_report_jg)



Validation Accuracy: 0.6546699963069131
Validation Classification Report:
               precision    recall  f1-score   support

           0       0.69      0.55      0.62     56794
           1       0.63      0.76      0.69     56932

    accuracy                           0.65    113726
   macro avg       0.66      0.65      0.65    113726
weighted avg       0.66      0.65      0.65    113726



In [23]:
print(val_accuracy_jg)
print('---------------------------------------------------------')
print(val_classification_report_jg)

0.6546699963069131
---------------------------------------------------------
              precision    recall  f1-score   support

           0       0.69      0.55      0.62     56794
           1       0.63      0.76      0.69     56932

    accuracy                           0.65    113726
   macro avg       0.66      0.65      0.65    113726
weighted avg       0.66      0.65      0.65    113726

